## 0.0 setup

follow the setup instructions in `1_0_hybridization_setup.ipynb`

## 0.1. imports
### 0.1.1. regular imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

### 0.1.2. local imports

In [12]:
sys.path.append(os.path.join(Path.home(), 'pylcaio')) # required for local import of pylcaio
import pylcaio

## 0.2. file paths
### 0.2.1. directories

In [9]:
%%capture
print(path_dir_hybridization_input := os.path.join(Path.home(), 'hybridization_data/input'))
print(path_dir_hybridization_output := os.path.join(Path.home(), 'hybridization_data/output'))

### 0.2.2. databases

In [13]:
%%capture
# Exiobase
print(path_file_exiobase := os.path.join(path_dir_hybridization_output, 'exiobase_monetary_pxp_2012.pickle'))
# Ecoinvent
print(path_file_ecoinvent := os.path.join(path_dir_hybridization_output, 'ecoinvent_3_5_cutoffPandas_symmNorm.gz.pickle'))
# Hybrid
print(path_file_hybrid := os.path.join(path_dir_hybridization_output, 'hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.pickle'))

In [ ]:
path_hybrid_database: str = '~/data/hybrid_database.pickle'

## 1.1. `pylcaio` hybridization

load the hybrid database and instantiate an analysis object

In [ ]:
database_loader: pylcaio.DatabaseLoader  = pylcaio.DatabaseLoader(
    lca_database_processed = ecoinvent,
    io_database_processed = exiobase,
    lca_database_name_and_version = 'ecoinvent3.5',
    io_database_name_and_version = 'exiobase3'
)

In [ ]:
with open(path_pylcaio_database_loader_class_instance, 'wb') as file_handle:
    pickle.dump(obj = database_loader, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
lcaio_object: pylcaio.LCAIO = database_loader.combine_ecoinvent_exiobase(
    complete_extensions = False,
    impact_world = False,
    regionalized = False)

In [ ]:
with open(path_pylcaio_class_instance_before_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
lcaio_object.hybridize(
    price_neutral_cut_off_matrix = 'STAM',
    capitals = False,
    priceless_scaling = True)

In [ ]:
with open(path_file_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)